# Simulated Annealing and GSA for Feature Selection
## Simulated Annealing Algorithm) For Feature Selection on Churn Prediction
___

In [1]:
import sys
import os

# Get the directory of the current script or notebook
script_dir = os.path.dirname(os.path.abspath("__file__"))

# Construct the full path to the 'src' directory
src_dir = os.path.join(script_dir, '..', 'src')

# Append the 'src' directory to sys.path
sys.path.append(src_dir)

# Construct the full path to the 'data' directory
data_dir = os.path.join(script_dir, '..', 'data')

# Now, you can import the modules and access the data
from main import simulated_annealing
from GSA_implementation import GSA
#from benchmarks import F1 
from gsa_sa_iterative import gsa_sa_iterative
from generate_neighbor import generate_neighbor
# from utils import fitness_function, train_model
from utils3 import train_model


# Load your data (example)
import pandas as pd

# #dataset1
#df = pd.read_csv(os.path.join(data_dir, 'churn2.csv'))

# dataset2
df = pd.read_csv(os.path.join(data_dir, 'ChurnNormalize.csv'))

#dataset3
#df = pd.read_csv(os.path.join(data_dir, 'cell2cell.csv'))

#dataset4
# df = pd.read_csv(os.path.join(data_dir, 'telecom_churn.csv'))

#dataset5
#df = pd.read_csv(os.path.join(data_dir, 'cleaned_dataset5.csv'))

In [2]:
import sys
#sys.path.append(r'C:\Users\fakep\Documents\S3 ITS\Machine Learning with Python\Simulated-Annealing-Feature-Selection-main\src')
sys.path.append(r'C:\Users\hendr\Documents\S3 ITS\Machine Learning with Python\MachineLearning\Simulated-Annealing-Feature-Selection-main\src')

from main import simulated_annealing


In [3]:
import pandas as pd
import time  
import pandas as pd
import numpy as np
import random
from datetime import datetime as dt
from utils import train_model
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from GSA_implementation import GSA
from benchmarks import F1
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


#classifiers_name = "KNN"

#classifiers_name = {
#      "Random Forest": RandomForestClassifier(),
#      "SVM": SVC(),
#      "Decision Tree": DecisionTreeClassifier(),
#      "Logistic Regression": LogisticRegression(),
#      "Naive Bayes": GaussianNB(),
#      "KNN": KNeighborsClassifier(),
#      "Gradient Boosting": GradientBoostingClassifier(),
# }

In [5]:
# Load data
#df = pd.read_csv(r'C:\Users\fakep\Documents\S3 ITS\Machine Learning with Python\Simulated-Annealing-Feature-Selection-main\data\churn2.csv')
#df = pd.read_csv(r'C:\Users\hendr\Documents\S3 ITS\Machine Learning with Python\MachineLearning\Simulated-Annealing-Feature-Selection-main\data\churn2.csv')
# Drop the first column
#df = df.iloc[:, 1:]

#dataset1
# Assuming the target variable is named 'target' and all others are features
# X_train = df.drop(columns=['churn'])
# X_train = df.iloc[:, 1:-1] 
# X_train = df.drop(columns=['Unnamed: 0', 'churn'])
# y_train = df['churn']

#dataset2
X_train = df.drop(columns=['Churn'])
y_train = df['Churn']

#dataset3 & 4
# X_train = df.drop(columns=['Churn'])
# y_train = df['Churn']

#dataset5
#X = df.drop(columns=['churn'])
#y = df['churn']

#print(X_train.head())
#print(y_train.head())

In [6]:
X_train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,1,0,0,1,0.013889,0,0,1,2,1,2,2,2,2,2,1,3,0.115423,0.003437
1,0,0,1,1,0.472222,1,1,1,1,2,1,2,2,2,1,0,2,0.385075,0.217564
2,0,0,1,1,0.027778,1,1,1,1,1,2,2,2,2,2,1,2,0.354229,0.012453
3,0,0,1,1,0.625000,0,0,1,1,2,1,1,2,2,1,0,1,0.239303,0.211951
4,1,0,1,1,0.027778,1,1,2,2,2,2,2,2,2,2,1,3,0.521891,0.017462


In [ ]:
df.head()

In [8]:
#mencoba algoritma baru2 (10-11-2024)
#pakai yang ini

import pandas as pd
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from utils import get_classifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from time import perf_counter

#Define parameters for GSA and SA
lb = [0] * X_train.shape[1]
ub = [1] * X_train.shape[1]
dim = len(lb)
PopSize = 25  # Number of particles in GSA
gsa_iters = 75  # Number of GSA iterations per SA step
sa_iters =  10 #50  # Total number of SA steps
max_iters = 50  # Max iterations within each SA refinement
sa_alpha = 0.93  # Cooling schedule parameter for SA
convergence_threshold = 0.001  # Threshold for SA early stopping
thr = 0.15
num_runs = 1 # Number of independent runs per classifier for robustness


# Define the classifiers dictionary
# classifiers = {
#     "KNN": KNeighborsClassifier(n_neighbors=5, metric='euclidean'),
#     "SVM": SVC(probability=True),
#     "Random Forest": RandomForestClassifier(),
#     "Logistic Regression": LogisticRegression(),
#     "Naive Bayes": GaussianNB(),
#     "Gradient Boosting": GradientBoostingClassifier()
# }

classifiers = {
    "KNN": KNeighborsClassifier(n_neighbors=5, metric='euclidean'),
#     "SVM": SVC(probability=True),
#    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
#     "Logistic Regression": LogisticRegression(),
#     "Naive Bayes": GaussianNB(),
#     "Gradient Boosting": GradientBoostingClassifier()
}

# Initialize DataFrames to store results
all_results = pd.DataFrame()
#avg_and_best_results = pd.DataFrame()

# Loop through classifiers
for name, clf in classifiers.items():
    #classifier_results = []
    summary_rows = [] # will store one row per SA Iteration (treated as a "run")
    
    for run in range(1, num_runs + 1):
        print(f"Running Adaptive GSA-SA with {name} (Run {run})...")
        t0 = perf_counter()
        # Run Adaptive GSA-SA for feature selection
        best_solution, results, run_elapsed = gsa_sa_iterative(
            X_train, y_train, max_iters=max_iters,
            gsa_iters=gsa_iters, sa_iters=sa_iters,
            sa_alpha=sa_alpha, 
            convergence_threshold=convergence_threshold,
            clf_name=name
        )
        
        t1 = perf_counter()
        

        # Convert results to DataFrame
        results_df = pd.DataFrame(results)

       
        # Add evaluation metrics for each subset of features (using 3-fold CV)
        kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scoring = {
            "accuracy":  "accuracy",
            "precision": make_scorer(precision_score, zero_division=0),
            "recall":    make_scorer(recall_score,    zero_division=0),
            "f1":        make_scorer(f1_score,        zero_division=0),
            "roc_auc":   "roc_auc",
        }


        for idx, row in results_df.iterrows():
            subset_features = row["Feature Set"]              # list of column indices
            Xs = X_train.iloc[:, subset_features].values      # use numpy array for speed
            ys = y_train.values.ravel()

            # IMPORTANT: clone a fresh classifier each time if you mutate clf elsewhere
            pipe = Pipeline([("scaler", StandardScaler()), ("clf", clf)])
            cvres = cross_validate(pipe, Xs, ys, cv=kf, scoring=scoring, n_jobs=-1, return_train_score=False)

            results_df.loc[idx, "Accuracy"]  = cvres["test_accuracy"].mean()
            results_df.loc[idx, "Precision"] = cvres["test_precision"].mean()
            results_df.loc[idx, "Recall"]    = cvres["test_recall"].mean()
            results_df.loc[idx, "F1 Score"]  = cvres["test_f1"].mean()
            results_df.loc[idx, "AUC Score"] = cvres["test_roc_auc"].mean()

        t2 = perf_counter()
        
        search_time = run_elapsed              # what gsa_sa_iterative measured
        eval_time   = t2 - t1                     # CV after the search
        wall_time   = t2 - t0                     # total end-to-end

        # Use *wall_time* in your summary (or store all three)
        results_df["Search Time (s)"] = search_time
        results_df["Eval Time (s)"]   = eval_time
        results_df["Wall Clock (s)"]  = wall_time
        

        # Add columns for classifier and run
        results_df["Classifier"] = name
        results_df["Run"] = run

        # Append results to main DataFrame
        all_results = pd.concat([all_results, results_df], ignore_index=True)
        
        # --- Summarize: treat each SA Iteration as ONE "run" (aligns with your old SA concept) ---
        # If you later set num_runs>1, switch grouping to ["Run","SA Iteration"] to keep outer runs separate
        group_keys = ["SA Iteration"] if num_runs == 1 else ["Run", "SA Iteration"]
        grouped = results_df.groupby(group_keys, as_index=False)
        
        
        def safe_mean(g, col):
            return g[col].mean() if (col in g.columns and len(g[col])) else float("nan")
        
        for keys, g in grouped:
            # Unpack keys cleanly
            if isinstance(keys, (list, tuple)):
                sai = int(keys[-1])
                outer = int(keys[0]) if len(keys) == 2 else run
            else:
                sai = int(keys)
                outer = run

                
            # Per-SA iteration search time: sum of inner iteration times
            sa_search_time = float(g["Run Time (s)"].sum()) if "Run Time (s)" in g.columns else float("nan")

            # If you still want to display eval + wall for each SA iteration,
            # you can distribute the one eval block across SA iterations:
            sa_eval_time = eval_time / sa_iters
            sa_wall_time = sa_search_time + sa_eval_time

#             summary_rows.append({
#                 "Classifier":       name,
#                 "Outer Run":        outer,                     # only matters if num_runs > 1
#                 "Run":              sai,                       # SA Iteration => "Run" index
#                 "Iterations":       int(len(g)),               # ~50 inner iterations
#                 "Accuracy":         safe_mean(g, "Accuracy"),
#                 "Precision":        safe_mean(g, "Precision"),
#                 "Recall":           safe_mean(g, "Recall"),
#                 "F1 Score":         safe_mean(g, "F1 Score"),
#                 "AUC":              safe_mean(g, "AUC Score"),
#                 "Selected F (avg)": safe_mean(g, "Feature Count"),
#                 # Use sum of per-iteration times if present; else fall back to wall-clock
#                 "Time (seconds)":   (g["Run Time (s)"].sum() if "Run Time (s)" in g.columns else run_elapsed)
#             })

            summary_rows.append({
                "Classifier":       name,
                "Outer Run":        outer,                     # only matters if num_runs > 1
                "Run":              sai,                       # SA Iteration => "Run" index
                "Iterations":       int(len(g)),               # ~50 inner iterations
                "Accuracy":         safe_mean(g, "Accuracy"),
                "Precision":        safe_mean(g, "Precision"),
                "Recall":           safe_mean(g, "Recall"),
                "F1 Score":         safe_mean(g, "F1 Score"),
                "AUC":              safe_mean(g, "AUC Score"),
                "Selected F (avg)": safe_mean(g, "Feature Count"),
                # ✅ Per-SA iteration timing (no more identical values)
                "Search Time (s)":  round(sa_search_time, 3),
                "Eval Time (s)":    round(sa_eval_time, 3),
                "Wall Clock (s)":   round(sa_wall_time, 3),
            })




    # ▼▼▼ STEP 3 (cont.): after run loop, write ONE CSV with one row per run ▼▼▼
#     summary_df = pd.DataFrame(summary_rows)
    summary_df = pd.DataFrame(summary_rows).sort_values(["Run"])
    timestamp  = datetime.now().strftime("%Y%m%d_%H%M%S")
#     summary_out = f"GRAS_summary_{name}_{timestamp}.csv"
#     summary_df.to_csv(summary_out, index=False)
#     print("Saved per-run summary:", summary_out)
    summary_out = f"GRAS_summary_{name}_by_SAIteration_{timestamp}.csv"
    summary_df.to_csv(summary_out, index=False)
    print("Saved per-SA-Iteration (per-run) summary:", summary_out)

    # (Optional) quick aggregate for the log
#     cols = ["Time (seconds)", "Accuracy", "Precision", "Recall", "F1 Score", "AUC", "Selected F (avg)"]

# (Optional) quick aggregate for the log
    cols = [
        "Search Time (s)", "Eval Time (s)", "Wall Clock (s)",
        "Accuracy", "Precision", "Recall", "F1 Score", "AUC", "Selected F (avg)"
        ]
   
    print("\nMean ± SD across runs:\n", summary_df[cols].agg(["mean", "std"]).round(4))    
    
    
    
    
# Define classifier names
classifier_names = "_".join(classifiers.keys())  # Combine classifier names into one string    
    
# Save results to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
all_results.to_csv(f"adaptive_gsa_sa_results_{classifier_names}_{num_runs}runs_{timestamp}.csv", index=False)
# avg_and_best_results.to_csv(f"adaptive_gsa_sa_average_best_metrics_{classifier_names}_{num_runs}_{timestamp}.csv", index=False)

print("Results saved successfully.")

Running Adaptive GSA-SA with KNN (Run 1)...
Starting SA Iteration 1
GSA is optimizing  "<lambda>"
[iter 1] G=10.0000
[iter 1] kbest=20
['At iteration 1 the best fitness is 0.23751210526315789']
[iter 2] G=9.3551
[iter 2] kbest=20
['At iteration 2 the best fitness is 0.2293415789473684']
[iter 3] G=8.7517
[iter 3] kbest=19
['At iteration 3 the best fitness is 0.2293415789473684']
['At iteration 4 the best fitness is 0.22457947368421052']
['At iteration 5 the best fitness is 0.22207315789473683']
['At iteration 6 the best fitness is 0.21956684210526312']
['At iteration 7 the best fitness is 0.21956684210526312']
['At iteration 8 the best fitness is 0.21752421052631576']
['At iteration 9 the best fitness is 0.21752421052631576']
['At iteration 10 the best fitness is 0.21560684210526312']
['At iteration 11 the best fitness is 0.21560684210526312']
['At iteration 12 the best fitness is 0.21560684210526312']
['At iteration 13 the best fitness is 0.21560684210526312']
['At iteration 14 the be

['At iteration 10 the best fitness is 0.2204315789473684']
['At iteration 11 the best fitness is 0.21752421052631576']
['At iteration 12 the best fitness is 0.21752421052631576']
['At iteration 13 the best fitness is 0.21752421052631576']
['At iteration 14 the best fitness is 0.21752421052631576']
['At iteration 15 the best fitness is 0.21752421052631576']
['At iteration 16 the best fitness is 0.21752421052631576']
['At iteration 17 the best fitness is 0.21752421052631576']
['At iteration 18 the best fitness is 0.21752421052631576']
['At iteration 19 the best fitness is 0.21752421052631576']
['At iteration 20 the best fitness is 0.21752421052631576']
['At iteration 21 the best fitness is 0.21752421052631576']
['At iteration 22 the best fitness is 0.21653421052631575']
['At iteration 23 the best fitness is 0.21653421052631575']
['At iteration 24 the best fitness is 0.21653421052631575']
['At iteration 25 the best fitness is 0.21653421052631575']
['At iteration 26 the best fitness is 0.2

['At iteration 23 the best fitness is 0.21752421052631576']
['At iteration 24 the best fitness is 0.21752421052631576']
['At iteration 25 the best fitness is 0.21752421052631576']
['At iteration 26 the best fitness is 0.21752421052631576']
['At iteration 27 the best fitness is 0.21752421052631576']
['At iteration 28 the best fitness is 0.21752421052631576']
['At iteration 29 the best fitness is 0.21752421052631576']
['At iteration 30 the best fitness is 0.21752421052631576']
['At iteration 31 the best fitness is 0.21752421052631576']
['At iteration 32 the best fitness is 0.21752421052631576']
['At iteration 33 the best fitness is 0.21752421052631576']
['At iteration 34 the best fitness is 0.21752421052631576']
['At iteration 35 the best fitness is 0.21752421052631576']
['At iteration 36 the best fitness is 0.21752421052631576']
['At iteration 37 the best fitness is 0.21752421052631576']
['At iteration 38 the best fitness is 0.21752421052631576']
['At iteration 39 the best fitness is 0.

['At iteration 36 the best fitness is 0.21455421052631576']
['At iteration 37 the best fitness is 0.21455421052631576']
['At iteration 38 the best fitness is 0.21455421052631576']
['At iteration 39 the best fitness is 0.21455421052631576']
['At iteration 40 the best fitness is 0.21455421052631576']
['At iteration 41 the best fitness is 0.21455421052631576']
['At iteration 42 the best fitness is 0.21455421052631576']
['At iteration 43 the best fitness is 0.21455421052631576']
['At iteration 44 the best fitness is 0.21455421052631576']
['At iteration 45 the best fitness is 0.21455421052631576']
['At iteration 46 the best fitness is 0.21455421052631576']
['At iteration 47 the best fitness is 0.21455421052631576']
['At iteration 48 the best fitness is 0.21455421052631576']
['At iteration 49 the best fitness is 0.21455421052631576']
['At iteration 50 the best fitness is 0.21455421052631576']
['At iteration 51 the best fitness is 0.21455421052631576']
['At iteration 52 the best fitness is 0.

['At iteration 49 the best fitness is 0.21402789473684208']
['At iteration 50 the best fitness is 0.21402789473684208']
['At iteration 51 the best fitness is 0.21402789473684208']
['At iteration 52 the best fitness is 0.21402789473684208']
['At iteration 53 the best fitness is 0.21402789473684208']
['At iteration 54 the best fitness is 0.21402789473684208']
['At iteration 55 the best fitness is 0.21402789473684208']
['At iteration 56 the best fitness is 0.21402789473684208']
['At iteration 57 the best fitness is 0.21402789473684208']
['At iteration 58 the best fitness is 0.21402789473684208']
['At iteration 59 the best fitness is 0.21402789473684208']
['At iteration 60 the best fitness is 0.21402789473684208']
['At iteration 61 the best fitness is 0.21402789473684208']
['At iteration 62 the best fitness is 0.21402789473684208']
['At iteration 63 the best fitness is 0.21402789473684208']
['At iteration 64 the best fitness is 0.21402789473684208']
['At iteration 65 the best fitness is 0.

C:\Users\hendr\AppData\Local\Temp\ipykernel_1373324\3390823431.py:136: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, g in grouped:


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['churn']),  # Features
    df['churn'],                # Target
    test_size=0.30,             # 30% for testing
    random_state=42             # For reproducibility
)


In [ ]:
#Mencoba algoritma baru (10-11-2024)

# Define parameters for GSA and SA
lb = [0] * X_train.shape[1]
ub = [1] * X_train.shape[1]
dim = len(lb)
PopSize = 30  # Number of particles in GSA
gsa_iters = 20  # Number of GSA iterations per SA step
sa_iters =  30 #50  # Total number of SA steps
max_iters = 30  # Max iterations within each SA refinement
alpha = 0.93  # Cooling schedule parameter for SA
convergence_threshold = 0.001  # Threshold for SA early stopping
num_runs = 1 # Number of independent runs per classifier for robustness





In [ ]:
#mencoba algoritma baru2 (10-11-2024)
#pakai yang ini

import pandas as pd
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from utils import get_classifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score

#Define parameters for GSA and SA
lb = [0] * X_train.shape[1]
ub = [1] * X_train.shape[1]
dim = len(lb)
PopSize = 20  # Number of particles in GSA
gsa_iters = 25  # Number of GSA iterations per SA step
sa_iters =  10 #50  # Total number of SA steps
max_iters = 50  # Max iterations within each SA refinement
alpha = 0.93  # Cooling schedule parameter for SA
convergence_threshold = 0.001  # Threshold for SA early stopping
num_runs = 1 # Number of independent runs per classifier for robustness


# Define the classifiers dictionary
# classifiers = {
#     "KNN": KNeighborsClassifier(n_neighbors=5, metric='euclidean'),
#     "SVM": SVC(probability=True),
#     "Random Forest": RandomForestClassifier(),
#     "Logistic Regression": LogisticRegression(),
#     "Naive Bayes": GaussianNB(),
#     "Gradient Boosting": GradientBoostingClassifier()
# }

classifiers = {
    "KNN": KNeighborsClassifier(n_neighbors=5, metric='euclidean'),
#     "SVM": SVC(probability=True),
#    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
#     "Logistic Regression": LogisticRegression(),
#     "Naive Bayes": GaussianNB(),
#     "Gradient Boosting": GradientBoostingClassifier()
}

# Initialize DataFrames to store results
all_results = pd.DataFrame()
#avg_and_best_results = pd.DataFrame()

# Loop through classifiers
for name, clf in classifiers.items():
    #classifier_results = []
    summary_rows = [] # will store one row per SA Iteration (treated as a "run")
    
    for run in range(1, num_runs + 1):
        print(f"Running Adaptive GSA-SA with {name} (Run {run})...")

        # Run Adaptive GSA-SA for feature selection
        best_solution, results, run_elapsed = gsa_sa_iterative(
            X_train, y_train, max_iters=max_iters,
            gsa_iters=gsa_iters, sa_iters=sa_iters,
            alpha=alpha, convergence_threshold=convergence_threshold,
            clf_name=name
        )

        # Convert results to DataFrame
        results_df = pd.DataFrame(results)

       
        # Add evaluation metrics for each subset of features (using 3-fold CV)
        kf = KFold(n_splits=3, shuffle=True, random_state=42)

        for idx, row in results_df.iterrows():
            subset_features = row["Feature Set"]              # list of column indices
            Xs = X_train.iloc[:, subset_features].values      # use numpy array for speed
            ys = y_train.values.ravel()

            # IMPORTANT: clone a fresh classifier each time if you mutate clf elsewhere
            model = clf

            acc = cross_val_score(model, Xs, ys, cv=kf, scoring="accuracy",  n_jobs=-1).mean()
            prec = cross_val_score(model, Xs, ys, cv=kf, scoring="precision", n_jobs=-1).mean()
            rec  = cross_val_score(model, Xs, ys, cv=kf, scoring="recall",    n_jobs=-1).mean()
            f1   = cross_val_score(model, Xs, ys, cv=kf, scoring="f1",        n_jobs=-1).mean()
            auc  = cross_val_score(model, Xs, ys, cv=kf, scoring="roc_auc",   n_jobs=-1).mean()

            results_df.loc[idx, "Accuracy"]  = acc
            results_df.loc[idx, "Precision"] = prec
            results_df.loc[idx, "Recall"]    = rec
            results_df.loc[idx, "F1 Score"]  = f1
            results_df.loc[idx, "AUC Score"] = auc


        # Add columns for classifier and run
        results_df["Classifier"] = name
        results_df["Run"] = run

        # Append results to main DataFrame
        all_results = pd.concat([all_results, results_df], ignore_index=True)
        
        # --- Summarize: treat each SA Iteration as ONE "run" (aligns with your old SA concept) ---
        # If you later set num_runs>1, switch grouping to ["Run","SA Iteration"] to keep outer runs separate
        group_keys = ["SA Iteration"] if num_runs == 1 else ["Run", "SA Iteration"]
        grouped = results_df.groupby(group_keys, as_index=False)
        
        
        def safe_mean(g, col):
            return g[col].mean() if (col in g.columns and len(g[col])) else float("nan")
        
        for keys, g in grouped:
            # Unpack keys cleanly
            if isinstance(keys, (list, tuple)):
                sai = int(keys[-1])
                outer = int(keys[0]) if len(keys) == 2 else run
            else:
                sai = int(keys)
                outer = run

            summary_rows.append({
                "Classifier":       name,
                "Outer Run":        outer,                     # only matters if num_runs > 1
                "Run":              sai,                       # SA Iteration => "Run" index
                "Iterations":       int(len(g)),               # ~50 inner iterations
                "Accuracy":         safe_mean(g, "Accuracy"),
                "Precision":        safe_mean(g, "Precision"),
                "Recall":           safe_mean(g, "Recall"),
                "F1 Score":         safe_mean(g, "F1 Score"),
                "AUC":              safe_mean(g, "AUC Score"),
                "Selected F (avg)": safe_mean(g, "Feature Count"),
                # Use sum of per-iteration times if present; else fall back to wall-clock
                "Time (seconds)":   (g["Run Time (s)"].sum() if "Run Time (s)" in g.columns else run_elapsed)
            })



    # ▼▼▼ STEP 3 (cont.): after run loop, write ONE CSV with one row per run ▼▼▼
#     summary_df = pd.DataFrame(summary_rows)
    summary_df = pd.DataFrame(summary_rows).sort_values(["Run"])
    timestamp  = datetime.now().strftime("%Y%m%d_%H%M%S")
#     summary_out = f"GRAS_summary_{name}_{timestamp}.csv"
#     summary_df.to_csv(summary_out, index=False)
#     print("Saved per-run summary:", summary_out)
    summary_out = f"GRAS_summary_{name}_by_SAIteration_{timestamp}.csv"
    summary_df.to_csv(summary_out, index=False)
    print("Saved per-SA-Iteration (per-run) summary:", summary_out)

    # (Optional) quick aggregate for the log
    cols = ["Time (seconds)", "Accuracy", "Precision", "Recall", "F1 Score", "AUC", "Selected F (avg)"]
    print("\nMean ± SD across runs:\n", summary_df[cols].agg(["mean", "std"]).round(4))    
    
    
    
    
# Define classifier names
classifier_names = "_".join(classifiers.keys())  # Combine classifier names into one string    
    
# Save results to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
all_results.to_csv(f"adaptive_gsa_sa_results_{classifier_names}_{num_runs}runs_{timestamp}.csv", index=False)
# avg_and_best_results.to_csv(f"adaptive_gsa_sa_average_best_metrics_{classifier_names}_{num_runs}_{timestamp}.csv", index=False)

print("Results saved successfully.")

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime

# Define the classifiers
classifiers = {
    "KNN": KNeighborsClassifier(n_neighbors=5, metric='euclidean'),
    "SVM": SVC(probability=True),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame()

# Initialize an empty DataFrame to store average and best metrics for each run
avg_and_best_results = pd.DataFrame()

# Number of runs
num_runs = 10

# Define parameters for GSA and SA
lb = [0] * X_train.shape[1]
ub = [1] * X_train.shape[1]
dim = len(lb)
PopSize = 30
iters = 100

# Generate a timestamp for dynamic file names
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Get classifier names for the file name
classifier_names = "_".join(classifiers.keys())

# Loop through each classifier
for name, clf in classifiers.items():
    classifier_results = []
    
    for run in range(1, num_runs + 1):
        #print(f"Running FS-SA with {name} (Run {run})...")
        print(f"Running FS-GSA_SA with {name} (Run {run})...")
        
        # Run the simulated annealing process with the current classifier
        #results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name=name, run_index=run)
        
        # Run GSA to get the best solution
        gsa_solution = GSA(objf=F1, lb=lb, ub=ub, dim=dim, PopSize=PopSize, iters=iters, df=df)
        initial_solution = gsa_solution.gBest

        # Run Simulated Annealing with the initial solution from GSA
        results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name = name, run_index = run, initial_solution=initial_solution)
        
        # Add columns for the classifier name and run number
        results['Classifier'] = name
        results['Run'] = run
        
        # Append the results to the combined DataFrame
        all_results = pd.concat([all_results, results], ignore_index=True)
        
        # Store the best metric from this run
        classifier_results.append(best_metric)
        
        # Calculate the average metrics for this run
        avg_metrics = results.mean()
        avg_metrics['Classifier'] = name
        avg_metrics['Run'] = run
        avg_metrics['Type'] = 'Average'

        # Store the best model's metrics
        best_metrics = pd.Series(best_metric)
        best_metrics['Classifier'] = name
        best_metrics['Run'] = run
        best_metrics['Type'] = 'Best Model'

        # Append to the DataFrame
        avg_and_best_results = pd.concat([avg_and_best_results, pd.DataFrame([avg_metrics]), pd.DataFrame([best_metrics])], ignore_index=True)

    # Calculate overall averages across all runs for this classifier
    overall_avg_metrics = avg_and_best_results[(avg_and_best_results['Classifier'] == name) & (avg_and_best_results['Type'] == 'Average')].mean(numeric_only=True)
    overall_avg_metrics['Classifier'] = name
    overall_avg_metrics['Run'] = 'Overall Average'
    overall_avg_metrics['Type'] = 'Average'

    overall_best_metrics = avg_and_best_results[(avg_and_best_results['Classifier'] == name) & (avg_and_best_results['Type'] == 'Best Model')].mean(numeric_only=True)
    overall_best_metrics['Classifier'] = name
    overall_best_metrics['Run'] = 'Overall Best'
    overall_best_metrics['Type'] = 'Best Model'

    # Append overall averages to the DataFrame
    avg_and_best_results = pd.concat([avg_and_best_results, pd.DataFrame([overall_avg_metrics]), pd.DataFrame([overall_best_metrics])], ignore_index=True)

# Dynamic file names
output_csv = f'fs_sa_results_{classifier_names}_{num_runs}runs_{timestamp}.csv'
average_output_csv = f'fs_sa_average_best_metrics_{classifier_names}_{num_runs}runs_{timestamp}.csv'

# Save the combined results to a CSV file
all_results.to_csv(output_csv, index=False)

# Save the average and best metrics to another CSV file
avg_and_best_results.to_csv(average_output_csv, index=False)

print(f"Detailed results saved to {output_csv}")
print(f"Average and best metrics saved to {average_output_csv}")


In [ ]:
results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name ="KNN")

In [ ]:
# Define parameters for GSA and SA
lb = [0] * X_train.shape[1]
ub = [1] * X_train.shape[1]
dim = len(lb)
PopSize = 30
iters = 100

# Run GSA to get the best solution
gsa_solution = GSA(objf=F1, lb=lb, ub=ub, dim=dim, PopSize=PopSize, iters=iters, df=df)
initial_solution = gsa_solution.gBest

# Run Simulated Annealing with the initial solution from GSA
results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, initial_solution=initial_solution)


In [ ]:
results

In [ ]:
best_metric

In [ ]:
best_subset_cols

In [ ]:
# Multiple Runs for Robustness Testing
num_runs = 10  # Number of runs
run_times = []
all_metrics = []
selected_features_count = []


classifier_name = "KNN"
fitness_function = "accuracy"


for run in range(num_runs):
    print(f"Starting Run {run + 1}")
    
    # Start the timer for the run
    start_time = time.time()
    
    # Run the simulated annealing process
    results, best_metric, best_subset_cols = simulated_annealing(X_train, y_train, classifier_name, fitness_function, run_index=run+1)
    
    # End the timer for the run
    end_time = time.time()
    
    # Calculate the run time and store it
    run_time = end_time - start_time
    run_times.append(run_time)
    
    print(f"Run {run + 1} completed in {run_time:.2f} seconds")
    
    
    #store the best metric from this run
    all_metrics.append(best_metric)
    # Optionally store the results
    # all_results.append((results, best_metric, best_subset_cols))
    
    # Store the number of features selected in this run
    selected_features_count.append(len(best_subset_cols))
    

# After all runs are complete
mean_runtime = np.mean(run_times)
std_runtime = np.std(run_times)

print(f"Average run time over {num_runs} runs: {mean_runtime:.2f} seconds")
print(f"Standard deviation of run time: {std_runtime:.2f} seconds")

# Calculate average and standard deviation of each metric
avg_metrics = {key: np.mean([metric[key] for metric in all_metrics]) for key in all_metrics[0]}
std_metrics = {key: np.std([metric[key] for metric in all_metrics]) for key in all_metrics[0]}

# Display average and standard deviation of metrics
print("\nAverage Metrics over all runs:")
for metric, value in avg_metrics.items():
    print(f"{metric}: {value:.3f}")

print("\nStandard Deviation of Metrics over all runs:")
for metric, value in std_metrics.items():
    print(f"{metric}: {value:.3f}")

# Display the best metrics for each run
print("\nBest Metrics from each run:")
for run_index, metrics in enumerate(all_metrics):
    print(f"Run {run_index + 1}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.3f}")

# Calculate and display the average number of features selected
average_features_selected = np.mean(selected_features_count)
print(f"\nAverage number of features selected over {num_runs} runs: {average_features_selected:.2f}")


In [ ]:
results